In [1]:
###### important Libraries  ######
import numpy as np 
import string
from nltk.corpus import stopwords
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from Levenshtein import ratio
from nltk.stem import PorterStemmer
pd.set_option('display.max_colwidth', None)
from textblob import Word
import random
import re
from tkinter import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk

In [5]:
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=550, height=650, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Welcome", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
    
    greeting_inputs = ("hey","Hello","hello", "Good Day", "hi", "whatsup")
    greeting_responses = [ "Hi..","Hello..", "hello", "Welcome", "Hey.."]

    def generate_greeting_response(self,greeting):
        for token in greeting.split():
            if token.lower() in self.greeting_inputs:
                return random.choice(self.greeting_responses)
    stop_words = stopwords.words('english')
    new_words=('explain','following','given','describe','ie','find','using','suppose','define','meaning','image','name','required','interview','discus''detail','indepth','short','brief','number','12','many','give','help','used','one','two','could','Algorithm','algorithm','basic','difference','respect','model')
    for i in new_words:
        stop_words.append(i)
            
    def get_response(self,user_input):
        print(user_input)
        data = pd.read_csv("Clean_supervised_learning.csv")
        sarthi_response = ''

        stop_words = stopwords.words('english')
        new_words=('explain','following','given','describe','ie','find','using','suppose','define','meaning','image','name','required','interview','discus''detail','indepth','short','brief','number','12','many','give','help','used','one','two','could','Algorithm','algorithm','basic','difference','respect','model')
        for i in new_words:
            stop_words.append(i)

        
        user_input=str(user_input)
        #user_input = user_input.replace('[^\w\s]','')
        user_input =re.sub(r'[^\w\s]', ' ', user_input)
        user_input = user_input.replace('-',' ')
        user_input = re.sub('[\?\.\/]',' ',user_input)
        user_input = user_input.split()
        user_input = (lambda user_input: [word.lower() for word in user_input])(user_input)
        user_input = (lambda user_input: [word for word in user_input if word not in stop_words])(user_input)
        stemmer=PorterStemmer()
        user_input = (lambda user_input:sorted([stemmer.stem(word) for word in user_input]))(user_input)
        wordnet_lemmatizer = WordNetLemmatizer()
        user_input = (lambda user_input:" ".join(sorted([wordnet_lemmatizer.lemmatize(word) for word in user_input])))(user_input)
        #print(user_input)

        mel_count=data["question_clean"].str.contains(user_input).sum()
        if mel_count==0:
            sarthi_response = sarthi_response + "I am sorry, I could not understand you."
            return sarthi_response
        else:
            for idx, row in data.iterrows():
                score = ratio(row["question_clean"], user_input)
                if score >= 0.90: # I'm sure, stop here
                    return row["Answer"]
            else:
                f_ans=sarthi_response + "So sorry, I am designed to discuss on General and supervised machine learning only."

            sarthi_response=f_ans
            return sarthi_response
    
    
    
        
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        bot_name = "Sarthi"
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg = msg.lower()
        if msg != 'bye':
            if msg == 'thanks' or msg == 'thank you very much' or msg == 'thank you':
                mw="Most welcome"
                msg2=f"{bot_name}: {mw}\n\n"
            else:
                if self.generate_greeting_response(msg) != None:
                    msg2= f"{bot_name}: {self.generate_greeting_response(msg)}\n\n"
                else:

                    msg2 = f"{bot_name}: {self.get_response(msg)}\n\n"

        else:
            bye="Good bye and take care of yourself..."
            msg2= f"{bot_name}: {bye}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)

        self.text_widget.see(END)
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()
    

what is linear regression algorithm ?
what is data science ?
